In [1]:
import nltk

# nltk.download('stopwords')

C:\ProgramData\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

In [3]:
# Gensim (Topic Modeling Pacakge)
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

In [4]:
# spacy for lemmatization
import spacy

from spacy_download import load_spacy

In [5]:
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [7]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [8]:
'''
We will be using the 20-Newsgroups dataset for this exercise.
This version of the dataset contains about 11k newsgroups posts from 
20 different topics. This is available as newsgroups.json.
'''
# Import Dataset
df = pd.read_csv('post_covidTweets.csv')
print(df.username.unique())
df.head()
df.count()

['AndyVermaut' 'nicktompan' 'yasu_yasuno_sa' 'islandofcovid' 'COVIDLive'
 'latimes' 'newsfilterio' 'VonPyotr' 'tazalot1981' 'SharonShepshan'
 'Mott_Mason' 'Ravie777' 'CTVNews' 'pop_popculture' 'TraderMarcoCost'
 'VivekSubbiah' 'boatratbown' 'mlauriat' 'LouWasHe' 'meer10001' 'elvisjj'
 'NZStuff' 'Alan_Nishihara' 'NuritBaytch' 'jonathanrockoff' 'WatchOurCity'
 'alicuff' 'Su15100307' 'bmangh' 'tjmakiboi' 'roto_rua' 'usatodaysun21'
 'CassieY4' 'Reuters' 'DennisKendel' 'thedextazlab' 'fox5dc' 'caa1000'
 'TweeterSomebody' 'ErieNewsNow' 'MelanieMoore' 'wbaltv11' 'easyOntario'
 'SocialistVoice' 'mcbc' 'US_FDA' 'Okie_Resister' 'sapp_erlot'
 'apokerplayer' 'alexselby1770' 'corndnc1' 'DrAnthony' '1NewsNZ'
 'DrEricDing' 'HammerToe' 'AstorAaron' 'openletterbot' 'NedPagliarulo'
 'helenbinchy' 'Teelin' 'KodyKinsley' 'thehill' 'ReginaPhelps'
 'TheHinduScience' 'JulianneMcShane' 'RavenHawk4' 'drpestuk' 'sauceocho'
 'TALK1370' '509ShawnG' 'PulpNews' 'chuckghunter' 'MHGtweet'
 'MarianneSkolek' 'dubvNOW' 

datetime          20560
tweet_id          20560
text              20560
username          20560
url               20560
user_location     18094
like_count        20560
retweet_count     20560
follower_count    20560
reply_count       20560
verified          20560
hashtags           2670
dtype: int64

In [9]:
'''
As you can see there are many emails, newline and extra spaces that is quite distracting.
Let’s get rid of them using regular expressions.
'''
# Convert to list
data = df.text.tolist()

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

#remove URL
data = [re.sub('http://\S+|https://\S+', '', sent) for sent in data]
data = [re.sub('http[s]?://\S+', '', sent) for sent in data]
data = [re.sub(r"http\S+", "", sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:1])

['Andy Vermaut shares:DOE finds COVID-19 pandemic possibly caused by '
 'accidental lab leak: Report | ABCNL Thank You. '
 '#AndyVermautFromBelgiumLovesABCnews #ThankYouForTheNews ']


In [10]:

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['andy', 'vermaut', 'shares', 'doe', 'finds', 'covid', 'pandemic', 'possibly', 'caused', 'by', 'accidental', 'lab', 'leak', 'report', 'abcnl', 'thank', 'you']]


In [11]:

# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['andy', 'vermaut', 'shares', 'doe', 'finds', 'covid', 'pandemic', 'possibly', 'caused', 'by', 'accidental', 'lab_leak', 'report', 'abcnl', 'thank', 'you']


In [12]:

# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [13]:
# Will download the model if it isn't installed yet 
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])



print(data_lemmatized[:1])

[['share', 'find', 'covid', 'pandemic', 'possibly', 'cause']]


In [14]:
import gensim.corpora as corpora

# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1)]]


In [15]:
vocab = len(id2word.token2id)

pd.Series([t for tokens in data_words_bigrams for t in tokens if "_" in t]).unique()

array(['lab_leak', 'white_house', 'abc_news', 'national_institutes',
       'drugmaker_seagen', 'early_stage', 'multi_billion', 'gambling_hub',
       'mask_mandate', 'florida_gov', 'ron_desantis', 'immune_systems',
       'ill_prepared', 'zach_shallcross', 'link_comments', 'feb_xinhua',
       'three_years', 'ensure_adequate', 'german_partner',
       'european_unions', 'likely_originated', 'created_stress',
       'dietary_habits', 'heaven_hell', 'successfully_meditation',
       'better_protection', 'stand_divided', 'additional_protection',
       'much_salt', 'immune_system', 'easy_prospective',
       'stds_similarities', 'loves_america', 'americans_feel',
       'packaged_fast', 'food_pollutants', 'obesity_going', 'us_folks',
       'vitamin_nature', 'late_stage', 'tracker_tokyo', 'border_controls',
       'upon_arrival', 'energy_department', 'laboratory_leak',
       'shortage_worsens', 'famine_yet', 'associated_press', 'en_espanol',
       'mindful_awareness', 'maximizing_natur

In [16]:

# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [17]:

# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]



[(0,
  '0.447*"death" + 0.138*"variant" + 0.069*"find" + 0.053*"bad" + '
  '0.050*"continue" + 0.046*"fall" + 0.027*"war" + 0.026*"safe" + '
  '0.019*"authority" + 0.013*"exercise"'),
 (1,
  '0.259*"life" + 0.094*"food" + 0.000*"save" + 0.000*"protect" + 0.000*"die" '
  '+ 0.000*"drug_administration" + 0.000*"step" + 0.000*"go" + 0.000*"barron" '
  '+ 0.000*"wave"'),
 (2,
  '0.190*"face" + 0.110*"immune_system" + 0.109*"school" + 0.093*"real" + '
  '0.031*"nature" + 0.021*"screen" + 0.007*"quick" + 0.001*"yield" + '
  '0.000*"additional_protection" + 0.000*"ecosearch"'),
 (3,
  '0.412*"people" + 0.304*"report" + 0.138*"claim" + 0.005*"expose" + '
  '0.000*"ecosearch" + 0.000*"die" + 0.000*"long" + 0.000*"hospital" + '
  '0.000*"average" + 0.000*"know"'),
 (4,
  '0.228*"infection" + 0.154*"ease" + 0.144*"virus" + 0.090*"state" + '
  '0.085*"symptom" + 0.046*"urge" + 0.042*"push" + 0.030*"affect" + '
  '0.019*"origin" + 0.015*"speak"'),
 (5,
  '0.000*"deeply" + 0.000*"crony" + 0.000*"bit

In [18]:
'''
Model perplexity and topic coherence provide a convenient measure to judge 
how good a given topic model is. In my experience, topic coherence score, 
in particular, has been more helpful.
'''
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)




Perplexity:  -18.413481086807007

Coherence Score:  0.3710010526872043


In [19]:
'''
Now that the LDA model is built, the next step is to examine the produced 
topics and the associated keywords. There is no better tool than pyLDAvis package’s 
interactive chart and is designed to work well with jupyter notebooks.
'''
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

C:\Users\hannah.kim\AppData\Roaming\Python\Python39\site-packages\pyLDAvis\_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
17     0.330223 -0.356003       1        1  24.982426
12     0.163142  0.186527       2        1   9.103481
10     0.147338  0.132211       3        1   8.535050
13     0.157917  0.160699       4        1   6.858104
11     0.092775  0.041797       5        1   5.902331
2     -0.159189 -0.032959       6        1   5.317544
0      0.095782  0.045690       7        1   5.299505
6     -0.080845 -0.028812       8        1   4.709470
9      0.037246  0.002241       9        1   4.399816
18     0.063947  0.018906      10        1   4.123018
16     0.016196 -0.006229      11        1   4.100464
4      0.032130  0.001006      12        1   4.068878
7      0.010259 -0.008019      13        1   3.595923
3      0.029902  0.000825      14        1   3.475392
15    -0.035003 -0.020518      15        1   3.305302
8     -0.154800 -0.033167      16        1   1.291487
1     -0.181710 -0.029438      17        1   0.438209
14    -0.185625 -0.027671      18        1   0.414138
5     -0.189844 -0.023543      19        1   0.039865
19    -0.189844 -0.023543      20        1   0.039598, topic_info=         Term          Freq         Total Category  logprob  loglift
1       covid  45159.000000  45159.000000  Default  30.0000  30.0000
119   vaccine   9871.000000   9871.000000  Default  29.0000  29.0000
52        new   5079.000000   5079.000000  Default  28.0000  28.0000
17     health   4859.000000   4859.000000  Default  27.0000  27.0000
178     death   4565.000000   4565.000000  Default  26.0000  26.0000
..        ...           ...           ...      ...      ...      ...
25    acquire      0.007626      2.393323  Topic20  -9.2117   2.0853
26   advanced      0.007626      2.393275  Topic20  -9.2117   2.0853
27        buy      0.007626     92.279084  Topic20  -9.2117  -1.5668
28     cancer      0.007626    550.261928  Topic20  -9.2117  -3.3524
29       deal      0.007626    148.950087  Topic20  -9.2117  -2.0456

[1009 rows x 6 columns], token_table=      Topic      Freq                   Term
term                                        
255       9  0.995236                 accord
162       6  0.297365  additional_protection
120       9  0.988534                  adult
304      12  0.990175                 affect
316       9  0.975932                 agency
...     ...       ...                    ...
273      11  0.976040                  woman
46        5  0.997482                  world
276      13  0.653249                worried
73        5  0.998893                   year
180       6  0.872383                  yield

[188 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[18, 13, 11, 14, 12, 3, 1, 7, 10, 19, 17, 5, 8, 4, 16, 9, 2, 15, 6, 20])